In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

!pip install tensorboardcolab
import tensorboardcolab

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.


# Load Cifar10 

In [2]:
## load cifar10 dataset 
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## reshape (None, 1) -> (None)
y_train, y_test = [y.reshape([-1]) for y in [y_train, y_test]]

## normalization 
x_train, x_test = [x for x in [x_train, x_test]]

## N classㅡ
n_classes = 10

In [4]:
y_test.shape

(10000,)

# Graph

In [3]:
data_dir = './'
path_pb = 'classify_image_graph_def.pb'
path = os.path.join(data_dir, path_pb)
graph = tf.Graph()

with graph.as_default() :
    #file 주소값
    with tf.gfile.FastGFile(path, 'rb') as file:
        graph_def  = tf.GraphDef()

        #string으로 read함
        graph_def.ParseFromString(file.read())
        tf.import_graph_def(graph_def, name = '')
    
    #InputTensor
    DecodeJpeg = graph.get_tensor_by_name("DecodeJpeg:0")

    #Get tensor pool_3/_reshape from models
    flat_layer = graph.get_tensor_by_name("pool_3/_reshape:0")

Instructions for updating:
Use tf.gfile.GFile.


# Generate cache

In [4]:
#extracted_feature
import sys 
def show_progess(n_total, index):
    sys.stdout.write('\r Progess {}/{}'.format(n_total, index))
    sys.stdout.flush()

def extract_cache(graph, sess, DecodeJpeg, flat_layer, xs):
    with graph.as_default() :  
        caches = [] 
        for ind, x in enumerate(xs):
            show_progess(len(xs), ind)
            feed_dict = {DecodeJpeg : x}
            cache = sess.run(flat_layer, feed_dict)
            cache = np.squeeze(cache)
            caches.append(cache)

        train_caches = np.stack(caches)
        return train_caches

# Open Session

In [5]:
sess = tf.Session(graph=graph)

# Extract Cache from Test Dataset

In [6]:
test_caches = extract_cache(graph, sess, DecodeJpeg, flat_layer, x_test[:])

 Progess 10000/9999

# Extract Cache from Train Dataset

In [7]:
train_caches = extract_cache(graph, sess, DecodeJpeg, flat_layer, x_train[:])

 Progess 50000/49999

In [11]:
np.save('test_caches', test_caches)
np.save('train_caches', train_caches)

In [12]:
print(test_caches.shape)
print(train_caches.shape)

(10000, 2048)
(50000, 2048)
